## 💠 **Parte 2: Olist**

**Desafio:** Criar um notebook de análise exploratória (como fizemos na nossa aula de hoje) com todas as etapas de coleta, limpeza, análise e visualização com base de dados da Olist.

**Requisitos:**

    🔸 Formular no mínimo **2 perguntas** (caso queira se desafiar crie pelo menos mais perguntas além das 2 obrigatórias) para responder com suas análises;
    🔸 Utilizar pelo menos **3 bases de dados da Olist** (caso você deseje criar sua base do zero). Caso deseje continuar utilizando a que criamos em aula, é necessário incluir pelo menos mais **1 tabela** para enriquecer sua análise.

**Para responder as perguntas elaboradas usar:**

    🔸 Criar pelo menos **2 gráficos**.
    🔸 Exporte sua base final em csv.
◾ Submeta uma pasta que contenha:
    🔸 o arquivo **seu_nome.ipynb** com sua análise exploratória;
    🔸 a base final criada por você no formato .csv;

**Dicas:**

    🔸 Use os conhecimentos da aula para realizar a análise e as visualizações.
    🔸 Use as funções do Pandas para manipular os dados.
    🔸 Seja criativa e explore os dados para encontrar insights interessantes!

**Boa sorte!**

# Perguntas para análise:

* 1 - Qual o ticket médio diário,mensal e anual?
* 2 - Qual o ano teve o maior número de vendas em valor?
* 3 - Qual estado teve mais vendas?
* 4 - Pedidos cancelados por estado?
* 5 - Qual o produto mais vendido ?
* 6 - Qual vendedor com mais vendas ?

In [50]:
#importando as biblitecas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
#lendo o arquivo csv df pedidos

df_pedidos = pd.read_csv('../para-sala/dados/olist_orders_dataset.csv')
df_pedidos

In [ ]:
# lendo o arquivo csv df itens pedido
df_itens_pedido = pd.read_csv('../para-sala/dados/olist_order_items_dataset.csv')
df_itens_pedido

In [ ]:
#Fazendo o merge para juntar os dfs usando a função merge e inner com base nas colunas 'order_id'
df_olist_pedidos = pd.merge(df_pedidos, df_itens_pedido, on=['order_id'], how='inner')
df_olist_pedidos

In [ ]:
#lendo o arquivo csv df consumidor
df_consumidor = pd.read_csv('../para-sala/dados/olist_customers_dataset.csv')
df_consumidor

In [ ]:
# juntando os  dois dfs, com base na colunas'customer_id'.
df_olist_pedidos_uf = pd.merge(df_olist_pedidos, df_consumidor, on=['customer_id'], how='inner')
df_olist_pedidos_uf

# * Iniciando a limpeza dos dados

In [ ]:
# Verificando as informações dos dados
df_pedidos.info()

In [ ]:
# criando uma lista com a colunas que não seram utilizadas e excluindo-as do df
columns_drop = ['order_approved_at', 'order_delivered_carrier_date', 'order_estimated_delivery_date']
df_pedidos = df_pedidos.drop(columns_drop, axis=1)
df_pedidos

In [ ]:
# Verificando os status disponíveis no df
df_pedidos['order_status'].unique()

In [ ]:
# Verificando a quantidade de pedidos finalizados
df_pedidos[df_pedidos['order_status'] == 'delivered']

In [ ]:
# Verificando a quantidade de  pedidos cancelados
df_pedidos[df_pedidos['order_status'] == 'canceled']



In [ ]:
# Filtrando somente pedidos finalizados, percebemos que perdemos um número razoável de linhas, cerca de 3 mil.
# Agora, com nosso dataset filtrado, precisamos resetar o seu index:
df_pedidos = df_pedidos.reset_index(drop=True)
df_pedidos

In [ ]:
df_pedidos.info()

In [ ]:
# transformando a colunas de datas que estão como string em datatime: 
df_pedidos['order_purchase_timestamp'] = pd.to_datetime(df_pedidos['order_purchase_timestamp'])
df_pedidos['order_delivered_customer_date'] = pd.to_datetime(df_pedidos['order_delivered_customer_date'])

#Criando a colunas com mês e ano por que iremmos usar para nossa análise
df_pedidos['reference_month_year'] = pd.to_datetime(df_pedidos['order_purchase_timestamp'].dt.strftime('%Y-%m'))
df_pedidos

In [ ]:
# verificando o dados df itens pedido
df_itens_pedido.info()

In [ ]:
# Verificando os dados df consumidor
df_consumidor.info()

In [ ]:
# Excluindo a coluna 'customer_unique_id'
df_consumidor = df_consumidor.drop('customer_unique_id', axis=1)
df_consumidor

In [ ]:
# Por padrão é realizado o merge dos dados com o inner join
# Jutando os df pedidos e itens de pedidos
df_olist = pd.merge(df_pedidos, df_itens_pedido, on='order_id', how='inner')
df_olist

In [ ]:
# Jutando agora com os dados de consumidor
df_olist = pd.merge(df_olist, df_consumidor, on='customer_id', how='inner')
df_olist

In [ ]:
df_olist.info()

In [ ]:
# Aqui estamos buscando uma estatística descritiva, podemos utilizar a nossa função describe() para trazer essas informações
df_olist.describe().T

In [71]:
# Exportando a base final em csv
df_olist.to_csv('Base_final_2_olist.csv',index=False)

# Perguntas :
* 1 - Qual o ticket médio diário,mensal e anual?
* 2 - Qual o ano teve o maior número de vendas em valor?
* 3 - Qual estado teve mais vendas?
* 4 - Pedidos cancelados por estado?
* 5 - Qual o produto mais vendido ?
* 6 - Qual vendedor com mais vendas ?

# 1 - Qual o ticket médio diário,mensal e anual?

In [ ]:
#Verificando o ticket médio diário
ticket_médio_diario = df_olist['reference_month_year'].dt.day
ticket_médio_diario = df_olist.groupby(ticket_médio_diario)['price'].mean()
ticket_médio_diario

In [ ]:
#verificando o ticket média mensal
mes_filter = df_olist['reference_month_year'].dt.month
ticket_médio_mensal = df_olist.groupby(mes_filter)['price'].mean()
ticket_médio_mensal



In [ ]:
#verificando o ticket média anual
ticket_médio_anual = df_olist['reference_month_year'].dt.year
ticket_médio_anual = df_olist.groupby(ticket_médio_anual)['price'].mean()
ticket_médio_anual


# 2 - Qual o ano teve o maior número de vendas em valor?

In [ ]:
# Verificando total de vendas por ano para verificar o ano com mais vendas
vendas_anual = df_olist['reference_month_year'].dt.year
total_vendas_anual = df_olist.groupby(vendas_anual)['price'].sum()
total_vendas_anual


In [ ]:
# definir tamanho da figura
plt.figure(figsize=(8, 6))

# definindo uma lista de cores para o gráfico
cores = ['red', 'yellow', 'green']
# instanciar o tipo de plotagem, no caso, um countplot usando x como os anos
sns.countplot(data = df_olist,
              x = vendas_anual,
              palette=cores)

# definir texto do título e eixos x e y
plt.title('Total de vendas por ano')
plt.xlabel('Anos')
plt.ylabel('Contagem')
plt.show()

# 3 - Qual o estado teve mais vendas?

In [ ]:
#Verificando os estados
df_olist['customer_state'].unique()

In [ ]:
# verificando em qual estado teve mais vendas
regiao = df_olist['customer_state'].mode()
regiao


In [ ]:
# Filtrando as vendas finalizadas
vendas_finalizadas = df_olist[df_olist['order_status'] == 'delivered']

# Agrupando as vendas finalizadas por estado e contando a quantidade de vendas finalizadas
venda_regiao = vendas_finalizadas.groupby('customer_state').size().reset_index(name='total_vendas')

venda_regiao



In [ ]:
# definir tamanho da figura
plt.figure(figsize=(10, 6))

# instanciar o tipo de plotagem, no caso, um countplot usando x como os tipos status do pedido e estado
sns.countplot(data = vendas_finalizadas,
              x = 'customer_state', 
              hue = 'order_status',
              palette=['#006400'])

# como o texto do eixo x é longo, podemos rotacioná-lo para melhorar a visualização
plt.xticks(rotation=330)

# definir texto do título e eixos x e y

plt.title('vendas finalizadas por estado')
plt.xlabel('Estado')
plt.ylabel('Contagem')

# plotar e definir texto da legenda
plt.legend(title = 'order_status')


plt.show()

# 4 - Pedidos cancelados por estado?

In [ ]:
# Filtrando os pedidos cancelados
vendas_canceladas = df_olist[df_olist['order_status'] == 'canceled']

# Agrupa as vendas finalizadas por estado e o status do pedido
venda_canceladas_região = vendas_canceladas.groupby('customer_state').size().reset_index(name='total_vendas')

venda_canceladas_região


In [ ]:
# definir tamanho da figura
plt.figure(figsize=(10, 6))

# instanciar o tipo de plotagem, no caso, um countplot usando x como os tipos status do pedido e estado

sns.countplot(data = vendas_canceladas,
              x = 'customer_state', 
              hue = 'order_status',
            palette=['#FF6347'])

# como o texto do eixo x é longo, podemos rotacioná-lo para melhorar a visualização
plt.xticks(rotation=330)

# definir texto do título e eixos x e y

plt.title('Vendas canceladas por estado')
plt.xlabel('Estado')
plt.ylabel('Contagem')

# plotar e definir texto da legenda
plt.legend(title='order_status')

plt.show()

# 5 - Qual o produto mais vendido ?

In [ ]:
# Filtrando somente vendas finalizadas
pedidos_finalizados = df_olist[df_olist['order_status'] == 'delivered']

# Usando a função mode() para verificar qual o produto mais vendido
produto_mais_vendido = pedidos_finalizados['product_id'].mode()
produto_mais_vendido

In [ ]:
# Verificando quantos foarm vendidos

contagem_vendas = pedidos_finalizados[(pedidos_finalizados['product_id'] == 'aca2eb7d00ea1a7b8ebd4e68314663af')]
contagem_vendas

# 6 - Qual vendedor com mais vendas ?

In [ ]:
# Verificando o id do vendedor com mais vendas usando a função mode()
vendedor_mais_vendas = pedidos_finalizados['seller_id'].mode()
vendedor_mais_vendas

In [ ]:
# Verificando a quantidade de vendas do vendedor
contagem_vendendedor = pedidos_finalizados[(pedidos_finalizados['seller_id'] == '6560211a19b47992c3666cc44a7e94c0')]
contagem_vendendedor